In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import torch
import numpy as np
from transformers import DistilBertModel, DistilBertTokenizer

import warnings
warnings.filterwarnings("ignore")


In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
data = pd.read_csv(r'../../data/final/final_dataset.csv')
data.head()

,chunk_id,text,label,file_name
0,-8673800439001282759,Getting Started This chapter will be about get...,computer,progit.txt
1,4883106761844832254,If you are a graphic or web designer and want ...,computer,progit.txt
2,7784288206342625690,This approach is very common because it is so ...,computer,progit.txt
3,9169385614588855871,"To deal with this problem, Centralized V...",computer,progit.txt
4,-6925518383648415521,"However, this setup also has some serious down...",computer,progit.txt


In [4]:
data.drop(columns=['chunk_id','file_name'],inplace=True)

In [5]:
map = {'biology': 0, 'chemistry': 1, 'computer': 2, 'maths': 3, 'physics': 4, 'social sciences': 5}
data['label'] = data['label'].map(map)
data.isna().sum()

text     0
label    0
dtype: int64

In [6]:
train_val_df, test_df = train_test_split(data, test_size=0.2, random_state = 42)
train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state = 42)

In [7]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [2]:
# We're using a base model for re-training with the base model trained on depression dataset
labels = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['label']]
        
        # Setting the parameters for the BERT tokenizer with max length = 512 and converting them to pytorch tensors
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in tqdm(df['text'])]
        
    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [3]:
from torch import nn
from transformers import BertConfig, BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.1):
        
        super(BertClassifier, self).__init__()
        
        self.bert = DistilBertModel.from_pretrained("distilbert/distilbert-base-uncased")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 192)
        self.linear1 = nn.Linear(192, 64)
        self.linear2 = nn.Linear(64, 6)
        self.gelu = nn.GELU()

    def forward(self, input_id, mask):
        output_1 = self.bert(input_ids= input_id,attention_mask=mask)
        hidden_state = output_1[0]
        pooled_output = hidden_state[:, 0]
        dropout_output = self.dropout(pooled_output)
        
        # Adding a new hidden layer to the output of BERT model
        linear_output1 = self.linear(dropout_output)
        linear_output1 = self.gelu(linear_output1)
        linear_output1 = self.dropout(linear_output1)
        
        # Adding a new hidden layer to the output of BERT model
        linear_output2 = self.linear1(linear_output1)
        linear_output2 = self.gelu(linear_output2)
        linear_output2 = self.dropout(linear_output2)
        
        # Adding the sigmoid layer for the model output
        final_layer = self.linear2(linear_output2)
        
        return final_layer

In [10]:
from torch.optim import Adam
from tqdm.notebook import tqdm
import time

In [11]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0))
print(torch.cuda.current_device())

2.2.1+cu121
12.1
8902
NVIDIA GeForce RTX 3070
_CudaDeviceProperties(name='NVIDIA GeForce RTX 3070', major=8, minor=6, total_memory=8191MB, multi_processor_count=46)
0


In [12]:
%%time
torch.backends.cudnn.benchmark = True

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)
    
    train_dataloader = torch.utils.data.DataLoader(train, batch_size = 32, shuffle=False) #, num_workers=4)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size = 32) #, num_workers = 4)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()
            
    # model, optimizer, dataloader = accelerator.prepare(model, optimizer, train_dataloader)

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            count = 0
            print(f'EPOCH NUMBER {epoch_num}')
            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
                output = model(input_id, mask)

                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad(set_to_none = True)
                batch_loss.backward()
                optimizer.step()              
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in tqdm(val_dataloader):
                    count += 1
                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}'
            )
            
    return model
                  
EPOCHS = 5
model = BertClassifier()
LR = 1e-05
              
final_model = train(model, (train_df), (val_df), LR, EPOCHS)

  0%|          | 0/184480 [00:00<?, ?it/s]

  0%|          | 0/46121 [00:00<?, ?it/s]

EPOCH NUMBER 0


  0%|          | 0/5765 [00:00<?, ?it/s]

  0%|          | 0/1442 [00:00<?, ?it/s]

...

  0%|          | 0/1442 [00:00<?, ?it/s]

Epochs: 4 | Train Loss:  0.002                 | Train Accuracy:  0.982                 | Val Loss:  0.006                 | Val Accuracy:  0.950
EPOCH NUMBER 4


  0%|          | 0/5765 [00:00<?, ?it/s]

  0%|          | 0/1442 [00:00<?, ?it/s]

Epochs: 5 | Train Loss:  0.001                 | Train Accuracy:  0.989                 | Val Loss:  0.006                 | Val Accuracy:  0.950
CPU times: user 18h 19min 13s, sys: 1min 34s, total: 18h 20min 47s
Wall time: 18h 20min 7s


In [13]:
torch.save(final_model.state_dict(), '../../models/distilbert_model')